In [1]:
%reload_ext tensorboard
import numpy as np
import json
import tensorflow as tf
import os

import time
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import gym

import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(gpus[0], True)
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(gpus[0], 
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=9000)])
  except RuntimeError as e:
    print(e)
    
    
print("GPUs: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.experimental.list_logical_devices('GPU')

import tf_agents
from tf_agents.networks import categorical_q_network
from tf_agents.agents.categorical_dqn import categorical_dqn_agent
from tf_agents.environments import py_environment, parallel_py_environment, batched_py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts
from tf_agents.networks.q_network import QNetwork
from tf_agents.agents.dqn.dqn_agent import DqnAgent
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.metrics import tf_metrics
from tf_agents.drivers.dynamic_step_driver import DynamicStepDriver
from tf_agents.policies.random_tf_policy import RandomTFPolicy
from tf_agents.utils.common import function, element_wise_squared_loss
from tf_agents.eval.metric_utils import log_metrics
from tf_agents.policies import policy_saver
import logging

import tensorflow.keras as keras

tf.compat.v1.enable_v2_behavior()
import time
import json
import datetime
import copy
import shutil

#import imp
from scan_gym import envs
#imp.reload(envs)
import csv

seed=42
tf.random.set_seed(seed)
np.random.seed(seed)

GPUs:  1


In [2]:
current_path = os.getcwd()
params_file = os.path.join(current_path, 'params.json') 
pm=json.load(open(params_file))
run_label = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
data_log_path = os.path.join(current_path, 'generated_data/') 

In [3]:
data_paths = ['/home/pico/uni/romi/scanner_cube/arabidopsis_im_bigger/204',
              '/home/pico/uni/romi/scanner_cube/arabidopsis_im_bigger/204',
             '/home/pico/uni/romi/scanner_cube/arabidopsis_im_bigger/202',
             '/home/pico/uni/romi/scanner_cube/arabidopsis_im_bigger/202']

#data_paths = ['','','','']

envs = [suite_gym.load('ScannerEnv-v1',gym_kwargs={'dataset_path':dp}) 
        for dp in data_paths]

tf_env = tf_py_environment.TFPyEnvironment( batched_py_environment.BatchedPyEnvironment(envs,multithreading=True), isolation=True  )

WARNING - 2021-10-05 21:40:39,644 - tf_py_environment - Wrapped environment is executing in parallel.  Perhaps it is a BatchedPyEnvironment with multithreading=True, or it is a ParallelPyEnvironment.  This conflicts with the `isolation` arg passed to TFPyEnvironment: interactions with the wrapped environment are no longer guaranteed to happen in a common thread.  Environment: (<tf_agents.environments.batched_py_environment.BatchedPyEnvironment object at 0x7f32410c8fa0>,)


In [4]:
tf_env.observation_spec()

(BoundedTensorSpec(shape=(66, 68, 152), dtype=tf.float16, name='observation/tuple_0', minimum=array(-1., dtype=float16), maximum=array(1., dtype=float16)),
 BoundedTensorSpec(shape=(10,), dtype=tf.int32, name='observation/tuple_1', minimum=array(-1, dtype=int32), maximum=array(179, dtype=int32)))

In [5]:
tf_env.action_spec()

BoundedTensorSpec(shape=(), dtype=tf.int64, name='action', minimum=array(0), maximum=array(16))

In [6]:
def volume_layers():
    input_vol = keras.layers.Input(shape=(66,68,152))
    preprocessing = keras.layers.Reshape((66,68,152,1))(input_vol)
    preprocessing = keras.layers.Cropping3D(cropping=((0,2), (0,4), (0,24)))(preprocessing) #output none,64,64,128
    preprocessing = keras.layers.Lambda(lambda x: (tf.cast(x,np.float32)+1.) / 2.)(preprocessing) #normalize 0-1
    
    
    x = keras.layers.Conv3D(filters=32, kernel_size=5,strides=2, padding="same", activation="relu")(preprocessing)
    #x = keras.layers.MaxPool3D(pool_size=2)(x)
    #x = keras.layers.BatchNormalization()(x)
    
    x = keras.layers.Conv3D(filters=32, kernel_size=3,strides=2, padding="same", activation="relu")(x)
    #x = keras.layers.MaxPool3D(pool_size=2)(x)
    #x = keras.layers.BatchNormalization()(x)

    x = keras.layers.Conv3D(filters=64, kernel_size=3,strides=2,padding="same", activation="relu")(x)
    #x = keras.layers.MaxPool3D(pool_size=2)(x)
    #x = keras.layers.BatchNormalization()(x)
    
    x = keras.layers.Conv3D(filters=64, kernel_size=3,strides=2,padding="same", activation="relu")(x)
    #x = keras.layers.MaxPool3D(pool_size=2)(x)
   
    #x = keras.layers.Flatten()(x)
    x = keras.layers.GlobalAveragePooling3D()(x)
    
    #x = keras.layers.Dense(32)(x)
                                        
    model = keras.models.Model(inputs=input_vol,outputs=x)
    model.summary()
    return model
    
    
#scale range 0 to 1
oldmin = tf_env.observation_spec()[1].minimum
oldmax = tf_env.observation_spec()[1].maximum
    
def input_vect_layers():
    input_ = keras.layers.Input(shape=(10,))
    preprocessing = keras.layers.Lambda(lambda x: ((x-oldmin)*(1.- 0.)/(oldmax-oldmin)) + 0. )(input_)
    #x = keras.layers.Dense(32)(preprocessing)
    return keras.models.Model(inputs=input_,outputs=preprocessing)

In [7]:
#network
preprocessing_layers=(volume_layers(),input_vect_layers())
preprocessing_combiner = tf.keras.layers.Concatenate(axis=-1)
dense_l = pm['model']['fc_layer_params']
if len(dense_l) == 1:
    fc_layer_params = (dense_l[0],)
else:
    fc_layer_params = dense_l


categorical_q_net = categorical_q_network.CategoricalQNetwork(
tf_env.observation_spec(),
tf_env.action_spec(),
preprocessing_layers=preprocessing_layers,
preprocessing_combiner=preprocessing_combiner,
fc_layer_params=fc_layer_params,
num_atoms=pm['categorical_dqn']['n_atoms'])

WARNING - 2021-10-05 21:40:40,117 - encoding_network - Beware: Copying a layer that has already been built: 'model'.  This can lead to subtle bugs because the original layer's weights will not be used in the copy.
WARNING - 2021-10-05 21:40:40,147 - encoding_network - Beware: Copying a layer that has already been built: 'model_1'.  This can lead to subtle bugs because the original layer's weights will not be used in the copy.


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 66, 68, 152)]     0         
_________________________________________________________________
reshape (Reshape)            (None, 66, 68, 152, 1)    0         
_________________________________________________________________
cropping3d (Cropping3D)      (None, 64, 64, 128, 1)    0         
_________________________________________________________________
lambda (Lambda)              (None, 64, 64, 128, 1)    0         
_________________________________________________________________
conv3d (Conv3D)              (None, 32, 32, 64, 32)    4032      
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 16, 16, 32, 32)    27680     
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 8, 8, 16, 64)      55360 

In [8]:
#agent
train_step = tf.Variable(0)
#optimizer = keras.optimizers.RMSprop(lr=2.5e-4, rho=0.95, momentum=0.0,
#            epsilon=0.00001, centered=True)
'''lr_decay =  keras.optimizers.schedules.PolynomialDecay(
            initial_learning_rate=0.005, # initial ε
            decay_steps = pm['agent']['decay_steps'], 
            end_learning_rate=0.0005)
optimizer = keras.optimizers.Adam(learning_rate=lambda: lr_decay(train_step))'''


optimizer = keras.optimizers.Adam(learning_rate=pm['model']['learning_rate'])

epsilon_fn = keras.optimizers.schedules.PolynomialDecay(
            initial_learning_rate=1.0, # initial ε
            decay_steps = pm['agent']['decay_steps'], 
            end_learning_rate=0.02) # final ε

agent = categorical_dqn_agent.CategoricalDqnAgent(tf_env.time_step_spec(),
                tf_env.action_spec(),
                categorical_q_network=categorical_q_net,
                optimizer=optimizer,
                min_q_value=pm['categorical_dqn']['min_q_value'],
                max_q_value=pm['categorical_dqn']['max_q_value'],
                target_update_period=pm['agent']['target_update_period'],
                td_errors_loss_fn= element_wise_squared_loss, #keras.losses.Huber(reduction="none"),
                gamma=pm['agent']['gamma'], # discount factor
                train_step_counter=train_step,
                n_step_update =  pm['categorical_dqn']['n_step_update'],                                
                epsilon_greedy=lambda: epsilon_fn(train_step))
agent.initialize()

WARNING - 2021-10-05 21:40:43,407 - encoding_network - Beware: Copying a layer that has already been built: 'model'.  This can lead to subtle bugs because the original layer's weights will not be used in the copy.
WARNING - 2021-10-05 21:40:43,436 - encoding_network - Beware: Copying a layer that has already been built: 'model_1'.  This can lead to subtle bugs because the original layer's weights will not be used in the copy.


In [9]:
#Replay buffer
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec= agent.collect_data_spec,
    batch_size= tf_env.batch_size,
    max_length=pm['rbuffer']['max_length'])

In [10]:
#observer
#observer is just a function (or a callable object) that takes a trajectory argument,
#add_method() method (bound to the replay_buffer object) can be used as observer
replay_buffer_observer = replay_buffer.add_batch

In [11]:
#observer for training metrics
training_metrics = [
tf_metrics.NumberOfEpisodes(),
tf_metrics.AverageEpisodeLengthMetric(batch_size=len(envs)),
tf_metrics.EnvironmentSteps(),
tf_metrics.AverageReturnMetric(batch_size=len(envs)),
tf_metrics.MaxReturnMetric(batch_size=len(envs)),
tf_metrics.MinReturnMetric(batch_size=len(envs)),   
]

In [12]:
#custom observer
class ShowProgress:
    def __init__(self, total):
        self.counter = 0
        self.total = total
    def __call__(self, trajectory):
        if not trajectory.is_boundary():
            self.counter += 1
        if self.counter % 100 == 0:
            print("\r{}/{}".format(self.counter, self.total), end="")

In [13]:
#Collect Driver
update_period = pm['collect_driver']['num_steps'] # train the model every x steps
collect_driver = DynamicStepDriver(
    tf_env,
    agent.collect_policy,
    observers=[replay_buffer_observer] + training_metrics,
    num_steps=update_period) # collect x steps for each training iteration

#+ training_metrics,

In [14]:
# random policy driver to start filling the buffer
random_collect_policy = RandomTFPolicy(tf_env.time_step_spec(),
                        tf_env.action_spec())

ns = pm['rnd_policy']['num_steps']
init_driver = DynamicStepDriver(
            tf_env,
            random_collect_policy,
            observers=[replay_buffer.add_batch],
            num_steps=ns)
#, ShowProgress(ns)],
final_time_step, final_policy_state = init_driver.run()

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))


WARNING - 2021-10-05 21:40:44,666 - deprecation - From /home/pico/.local/lib/python3.8/site-packages/tf_agents/drivers/dynamic_step_driver.py:196: calling while_loop_v2 (from tensorflow.python.ops.control_flow_ops) with back_prop=False is deprecated and will be removed in a future version.
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))


In [15]:
#use buffer as tf API dataset ()
dataset = replay_buffer.as_dataset(
        sample_batch_size=pm['rbuffer']['sample_batch_size'],
        num_steps=pm['categorical_dqn']['n_step_update'] + 1,
        num_parallel_calls=3).prefetch(3)

Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


WARNING - 2021-10-05 21:41:06,237 - api - From /home/pico/anaconda3/envs/rl/lib/python3.8/site-packages/tensorflow/python/autograph/operators/control_flow.py:1218: ReplayBuffer.get_next (from tf_agents.replay_buffers.replay_buffer) is deprecated and will be removed in a future version.
Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


In [16]:
#convert main functions to tensorflow functions to speed up training
collect_driver.run = function(collect_driver.run)
agent.train = function(agent.train)

In [17]:
#@tf.function
def train_agent(n_iterations):
    #reset metrics
    for m in training_metrics:
        m.reset()
    time_step = None
    policy_state = ()#agent.collect_policy.get_initial_state(tf_env.batch_size)
    agent.train_step_counter.assign(0)
    iterator = iter(dataset)
    for iteration in range(n_iterations):
        time_step, policy_state = collect_driver.run(time_step, policy_state)
        trajectories, buffer_info = next(iterator)
        train_loss = agent.train(trajectories)
        print("\r{} loss:{:.5f} epsilon{}".format(iteration, train_loss.loss.numpy(),epsilon_fn(train_step)), end="")
        if iteration % 100 == 0:
            with train_summary_writer.as_default():
                #plot metrics
                for train_metric in training_metrics:
                    train_metric.tf_summaries(train_step=tf.cast(agent.train_step_counter,tf.int64), step_metrics=training_metrics[:])
                train_summary_writer.flush()
                
        if iteration % 100 == 0:
            with train_summary_writer.as_default():
                #plot train loss          
                tf.summary.scalar('train_loss', train_loss.loss.numpy(), step=tf.cast(agent.train_step_counter,tf.int64))
                #plot NN weights
                for layer in  categorical_q_net.layers:
                    for weight in layer.weights:
                        tf.summary.histogram(weight.name,weight,step=tf.cast(agent.train_step_counter,tf.int64))
                train_summary_writer.flush()

In [18]:
train_dir = os.path.join(data_log_path,"logs/",run_label)   
train_summary_writer = tf.summary.create_file_writer(
            train_dir, flush_millis=10000)
#train_summary_writer.set_as_default()

In [19]:
# Launch TensorBoard with objects in the log directory
# This should launch tensorboard in your browser, but you may not see your metadata.
#%tensorboard --logdir=logs --reload_interval=15

In [20]:
#tf.summary.scalar('avgreturn', training_metrics[3].result().numpy(), step=tf.cast(agent.train_step_counter,tf.int64))

In [21]:
%time train_agent(pm['misc']['n_iterations'])

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))


WARNING - 2021-10-05 21:41:19,919 - deprecation - From /home/pico/anaconda3/envs/rl/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:201: calling foldr_v2 (from tensorflow.python.ops.functional_ops) with back_prop=False is deprecated and will be removed in a future version.
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))


49999 loss:1.14443 epsilon0.019999999552965164CPU times: user 5h 36min 39s, sys: 15min 53s, total: 5h 52min 32s
Wall time: 2h 14min 22s


In [22]:
def run_episode(env,tf_env,policy):
    state = tf_env.reset()
    time_steps = 40
    for i in range(1,time_steps):
        action_step = policy.action(state)
        state = tf_env.step(action_step.action)
        #print(i,':',action_step.action.numpy(),state.observation[1].numpy()[0],state.step_type[0].numpy())  #,state.step_type.numpy()  )#,env.action2move(action_step.action,state.observation[0][-1].numpy()))#action,dir
        last_position = env.current_position
        if state.is_last():
            break
    return env.h

def test_envs(envs,tf_envs,policy):
    stats = []
    for  model in range(len(envs)):
        hs = []
        for i in range(180):
            h = run_episode(envs[model], tf_envs[model],policy)
            hs.append(h)
        stats.append(np.mean(hs,axis=0).astype('int'))
        print(model)
    return stats

def save_tests(path,stats,models):
    f = open(path,'w')
    writer = csv.writer(f)
    writer.writerow(["model","empty", "undefined", "solid"])
    for i in range(len(stats)):
        st= np.insert(stats[i],0,models[i])
        writer.writerow(st)
    total_mean = np.mean(stats,axis=0)
    writer.writerow(np.insert(total_mean,0,999).astype('int'))
    f.close()

In [23]:
models = [204,202]#,79,88,146,198,199]
envs = [suite_gym.load('ScannerEnv-v1',
        gym_kwargs={'dataset_path':os.path.join(current_path,'arabidopsis_im_bigger',str(x).zfill(3)),'init_pos_inc_rst':True}) for x in models]

tf_envs = [tf_py_environment.TFPyEnvironment( env ) for env in envs]


dest_dir = os.path.join(data_log_path,'tests')
if not os.path.exists(dest_dir):
    os.makedirs(dest_dir)

data = test_envs(envs,tf_envs,agent.policy)    
test_file = os.path.join(dest_dir,run_label + '.csv')
save_tests(test_file,data,models)
print(data)

0
1
[array([556889,  14910, 110376]), array([614425,  18609,  49141])]


In [24]:
#save model, parameters and code state
policy_dir = os.path.join(data_log_path,"policies", run_label)
tf_policy_saver = policy_saver.PolicySaver(agent.policy)
tf_policy_saver.save(policy_dir)

with open(os.path.join(data_log_path,"parameters", run_label+'.json'), 'w') as json_file:
  json.dump(pm, json_file)

src = os.path.join(current_path,"categorical_dqn_tf_agents.ipynb")
dst = os.path.join(data_log_path,"train_code", run_label+'.ipynb')
shutil.copyfile(src, dst)

src = os.path.join(current_path,"scan_gym/scan_gym/envs/ScannerEnv/scanner_env.py")
dst = os.path.join(data_log_path,"environment_code", run_label+'.py')
shutil.copyfile(src, dst)


WARNING - 2021-10-05 23:57:15,757 - save - Found untraced functions such as CategoricalQNetwork_layer_call_and_return_conditional_losses, CategoricalQNetwork_layer_call_fn, CategoricalQNetwork_layer_call_and_return_conditional_losses, CategoricalQNetwork_layer_call_fn, CategoricalQNetwork_layer_call_fn while saving (showing 5 of 35). These functions will not be directly callable after loading.
WARNING - 2021-10-05 23:57:15,794 - save - Found untraced functions such as CategoricalQNetwork_layer_call_and_return_conditional_losses, CategoricalQNetwork_layer_call_fn, CategoricalQNetwork_layer_call_and_return_conditional_losses, CategoricalQNetwork_layer_call_fn, CategoricalQNetwork_layer_call_fn while saving (showing 5 of 35). These functions will not be directly callable after loading.


INFO:tensorflow:Assets written to: /home/pico/uni/romi/scanner_cube/generated_data/policies/20211005-214038/assets


INFO - 2021-10-05 23:57:16,689 - builder_impl - Assets written to: /home/pico/uni/romi/scanner_cube/generated_data/policies/20211005-214038/assets


'/home/pico/uni/romi/scanner_cube/generated_data/environment_code/20211005-214038.py'

In [25]:
replay_buffer.num_frames()

<tf.Tensor: shape=(), dtype=int64, numpy=16000>

In [26]:
dp = '/home/pico/uni/romi/scanner_cube/arabidopsis_im_bigger/204'
env = suite_gym.load('ScannerEnv-v1',gym_kwargs={'dataset_path':dp})
test_env = tf_py_environment.TFPyEnvironment( env )

In [48]:
state = test_env.reset()
#print(state.observation[0].numpy())
cont = 0
time_steps = 18
#obs = deque(maxlen=time_steps)
#print('0:',action_step.action.numpy(),state.observation[1].numpy()[0],state.step_type[0].numpy())

for i in range(1,time_steps):
    action_step = agent.policy.action(state)#agent.policy.action(state)
    state = test_env.step(action_step.action)
    print(i,':',action_step.action.numpy(),state.observation[1].numpy()[0],state.step_type[0].numpy())  #,state.step_type.numpy()  )#,env.action2move(action_step.action,state.observation[0][-1].numpy()))#action,dir
    last_position = env.current_position
    if state.is_last():
        print('salio',cont)
        break
        
print(env.kept_abs_images)
print(sorted(env.kept_abs_images))
print(env.total_reward)

print(env.h)
print(env.kept_rel_images)
print(sorted(env.kept_rel_images))

1 : [2] [ 0  5 -1 -1 -1 -1 -1 -1 -1 -1] 1
2 : [2] [ 0  5 10 -1 -1 -1 -1 -1 -1 -1] 1
3 : [2] [ 0  5 10 15 -1 -1 -1 -1 -1 -1] 1
4 : [1] [ 0  5 10 15 18 -1 -1 -1 -1 -1] 1
5 : [2] [ 0  5 10 15 18 23 -1 -1 -1 -1] 1
6 : [1] [ 0  5 10 15 18 23 26 -1 -1 -1] 1
7 : [3] [ 0  5 10 15 18 23 26 37 -1 -1] 1
8 : [5] [ 0  5 10 15 18 23 26 37 70 -1] 1
9 : [6] [  0   5  10  15  18  23  26  37  70 115] 2
salio 0
[154, 159, 164, 169, 172, 177, 0, 11, 44, 89]
[0, 11, 44, 89, 154, 159, 164, 169, 172, 177]
8.837366666666666
[561633, 15594, 104949]
[0, 5, 10, 15, 18, 23, 26, 37, 70, 115]
[0, 5, 10, 15, 18, 23, 26, 37, 70, 115]


In [28]:
#batched_py_environment.BatchedPyEnvironment(envs=[suite.load(...) for _ in range(n)])

In [29]:
#tf_env.observation_spec()[1].maximum

In [30]:
random_collect_policy = RandomTFPolicy(tf_env.time_step_spec(),
                        tf_env.action_spec())

In [31]:
random_collect_policy.collect_data_spec

Trajectory(step_type=TensorSpec(shape=(), dtype=tf.int32, name='step_type'), observation=(BoundedTensorSpec(shape=(66, 68, 152), dtype=tf.float16, name='observation/tuple_0', minimum=array(-1., dtype=float16), maximum=array(1., dtype=float16)), BoundedTensorSpec(shape=(10,), dtype=tf.int32, name='observation/tuple_1', minimum=array(-1, dtype=int32), maximum=array(179, dtype=int32))), action=BoundedTensorSpec(shape=(), dtype=tf.int64, name='action', minimum=array(0), maximum=array(16)), policy_info=(), next_step_type=TensorSpec(shape=(), dtype=tf.int32, name='step_type'), reward=TensorSpec(shape=(), dtype=tf.float32, name='reward'), discount=BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)))

In [32]:
#q_net.layers[0].weights[0].shape

In [33]:
random_collect_policy.time_step_spec

TimeStep(step_type=TensorSpec(shape=(), dtype=tf.int32, name='step_type'), reward=TensorSpec(shape=(), dtype=tf.float32, name='reward'), discount=BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)), observation=(BoundedTensorSpec(shape=(66, 68, 152), dtype=tf.float16, name='observation/tuple_0', minimum=array(-1., dtype=float16), maximum=array(1., dtype=float16)), BoundedTensorSpec(shape=(10,), dtype=tf.int32, name='observation/tuple_1', minimum=array(-1, dtype=int32), maximum=array(179, dtype=int32))))

In [34]:
ns = pm['rnd_policy']['num_steps']
init_driver = DynamicStepDriver(
            tf_env,
            random_collect_policy,
            observers=[replay_buffer.add_batch],
            num_steps=ns)

In [35]:
init_driver

In [36]:
state = tf_env.reset()

In [37]:
state

TimeStep(step_type=<tf.Tensor: shape=(4,), dtype=int32, numpy=array([0, 0, 0, 0], dtype=int32)>, reward=<tf.Tensor: shape=(4,), dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>, discount=<tf.Tensor: shape=(4,), dtype=float32, numpy=array([1., 1., 1., 1.], dtype=float32)>, observation=(<tf.Tensor: shape=(4, 66, 68, 152), dtype=float16, numpy=
array([[[[-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.],
         ...,
         [ 1.,  1.,  1., ...,  1.,  1.,  1.],
         [ 1.,  1.,  1., ...,  1.,  1.,  1.],
         [ 1.,  1.,  1., ...,  1.,  1.,  1.]],

        [[-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.],
         ...,
         [ 1.,  1.,  1., ...,  1.,  1.,  1.],
         [ 1.,  1.,  1., ...,  1.,  1.,  1.],
         [ 1.,  1.,  1., ...,  1.,  1.,  1.]],

        [[-1., -1., -1., ..., -1., -1., -1.],
         [-1.,

In [38]:
state = tf_env.step(0)

TypeError: iteration over a 0-d array

In [ ]:
st = env.reset()

In [ ]:
st[0].shape

In [ ]:
st

In [ ]:
random_collect_policy.action(state)

In [ ]:
import tf_agents.utils
tf_agents.utils.nest_utils.get_outer_shape(state,  tf_env.time_step_spec())

In [ ]:
agent.training_data_spec

In [ ]:
tf_env.observation_spec()[1].minimum